In [1]:
import numpy as np
import cv2 as cv
import os

In [2]:
def show_image(name, img):
    cv.imshow(name, img)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [3]:
def scale_down(scalex,scaley,img):
    img = cv.resize(img, (0,0), fx=scalex, fy=scaley)
    return img

In [4]:
def find_color_values_using_trackbar(frame,first = False):

    frame_hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
 
    def nothing(x):
        pass

    cv.namedWindow("Trackbar") 
    cv.resizeWindow('Trackbar', 400, 200)
    cv.createTrackbar("LH", "Trackbar", 0, 255, nothing)
    cv.createTrackbar("LS", "Trackbar", 0, 255, nothing)
    cv.createTrackbar("LV", "Trackbar", 0, 255, nothing)
    cv.createTrackbar("UH", "Trackbar", 255, 255, nothing)
    cv.createTrackbar("US", "Trackbar", 255, 255, nothing)
    cv.createTrackbar("UV", "Trackbar", 255, 255, nothing)
    
    imgs = None
    while True:

        l_h = cv.getTrackbarPos("LH", "Trackbar")
        l_s = cv.getTrackbarPos("LS", "Trackbar")
        l_v = cv.getTrackbarPos("LV", "Trackbar")
        u_h = cv.getTrackbarPos("UH", "Trackbar")
        u_s = cv.getTrackbarPos("US", "Trackbar")
        u_v = cv.getTrackbarPos("UV", "Trackbar")


        l = np.array([l_h, l_s, l_v])
        u = np.array([u_h, u_s, u_v])
        mask_table_hsv = cv.inRange(frame_hsv, l, u)        

        res = cv.bitwise_and(frame, frame, mask=mask_table_hsv)
        
        # Resize the images before displaying
        if not first:
            frame_resized = scale_down(0.3, 0.3, frame)
            mask_table_hsv_resized = scale_down(0.3, 0.3, mask_table_hsv)
            res_resized = scale_down(0.3, 0.3, res) 
        else:
            frame_resized = frame
            mask_table_hsv_resized = mask_table_hsv
            res_resized = res
        cv.imshow("Frame", frame_resized)
        cv.imshow("Mask", mask_table_hsv_resized)
        cv.imshow("Res", res_resized)
        
        if cv.waitKey(25) & 0xFF == ord('q'):
                
                imgs = [frame, mask_table_hsv, res, l, u]
                break
    cv.destroyAllWindows()
    return imgs

In [5]:
# # img = cv.imread("/home/mircea/CAVA/OneDrive_2023-11-18/Laborator 7/CAVA-2023-TEMA1/imagini_auxiliare/02.jpg")
# img = cv.cvtColor(debug_list[0][1].copy(), cv.COLOR_GRAY2RGB)
# if img is not None:
#     imgs = find_color_values_using_trackbar(img,first=True)
# print(imgs)

In [6]:
try_boundary = [np.array([0, 0, 0]), np.array([255, 185, 255])]

In [7]:
try_boundary

[array([0, 0, 0]), array([255, 185, 255])]

In [8]:
extract_pieces_boundary = [np.array([ 77,   0, 222]), np.array([116, 122, 255])]

In [9]:
pieces_boundary = [np.array([ 0,  0, 64]), np.array([122, 255, 255])]

In [10]:
possible_boundaries = [[np.array([  0, 135, 109]), np.array([255, 255, 255])],[np.array([24,0,0]), np.array([255, 255, 255])],[np.array([ 93, 157,  74]), np.array([107, 255, 255])]]

In [11]:
extract_pieces_boundary

[array([ 77,   0, 222]), array([116, 122, 255])]

In [12]:
def to_hsv(img,boundaries= possible_boundaries[1]):
    low,high = boundaries
    img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    mask_yellow_hsv = cv.inRange(img_hsv, low,high)
    res_hsv = cv.bitwise_and(img, img, mask=mask_yellow_hsv)
    
    return res_hsv

In [13]:
def crop_image(image, no_pixels):
    # Tăierea imaginii
    h,w = image.shape[:2]
    cropped_image = image[no_pixels:h-no_pixels, no_pixels:w-no_pixels,:]
    return cropped_image

In [14]:
from cv2 import sort

def extract_board(img):
    img = scale_down(0.3,0.3,img)
    # show_image('img', img)
    image = img.copy()
    
    # to hsv
    image = to_hsv(image,possible_boundaries[1])
    # show_image('image', image)
    # extract board
    image = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    image_m_blur = cv.medianBlur(image,19)
    
    image_g_blur = cv.GaussianBlur(image_m_blur, (0, 0), 5) 
    image_sharpened = cv.addWeighted(image_m_blur, 1.2, image_g_blur, -0.8, 0)
    # Aplicarea filtrului Sobel
    sobelx = cv.Sobel(image_sharpened, cv.CV_64F, 1, 0, ksize=5)
    sobely = cv.Sobel(image_sharpened, cv.CV_64F, 0, 1, ksize=5)

    # Calculul magnitudinii gradientului
    magnitude = np.sqrt(sobelx**2 + sobely**2)

    # Conversia la tipul de date uint8 (0-255)
    magnitude = np.uint8(magnitude)
    
    aux = magnitude
    aux = cv.bilateralFilter(aux,20,75,75)
    _, thresh = cv.threshold(aux, 10, 150, cv.THRESH_BINARY)
    # show_image('thresh', thresh)
    
    kernel = np.ones((3, 3), np.uint8)
    thresh = cv.erode(thresh, kernel)
    # show_image('thresh', thresh)
    

    edges =  cv.Canny(thresh ,100,200)
    
    
    contours, _ = cv.findContours(edges,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    max_area = 0
    
    
    contours, _ = cv.findContours(edges,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv.contourArea, reverse=True)
    cv.drawContours(image=image.copy(), contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv.LINE_AA)

    max_area = 0
   
    for i in range(len(contours)):
        if(len(contours[i]) >3):
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis = 1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left

    image_copy = cv.cvtColor(image.copy(),cv.COLOR_GRAY2BGR)
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)
    
    
    
    width = 810
    height = 810
    
    
    puzzle = np.array([top_left,top_right,bottom_right,bottom_left], dtype = "float32")
    destination_of_puzzle = np.array([[0,0],[width,0],[width,height],[0,height]], dtype = "float32")

    M = cv.getPerspectiveTransform(puzzle,destination_of_puzzle)

    result = cv.warpPerspective(img, M, (width, height))
   
    
    no_pixels = 100
    board = crop_image(result, no_pixels)
    # show_image("board",board)
    
    return board

In [15]:
def extract_grid(board):

    aux = board.copy()
    # Specificarea culorii cu care să fie umplute marginile (culoarea de fundal)
    border_color = [0, 0,0]  # Exemplu: verde (BGR)

    # Specificarea dimensiunilor imaginii
    height, width, _ = aux.shape

    # Specificarea dimensiunilor marginilor
    top, bottom, left, right = 23, 23, 23, 23

    # Umplerea marginilor imaginii cu culoarea specificată
    aux[:top, :] = border_color  # Top
    aux[-bottom:, :] = border_color  # Bottom
    aux[:, :left] = border_color  # Left
    aux[:, -right:] = border_color  # Right
    
    aux = to_hsv(aux,pieces_boundary)
    aux = cv.cvtColor(aux,cv.COLOR_BGR2GRAY)

    blur = cv.bilateralFilter(aux,30,120,120)
    
    #25 si 180 e 98 %
    _,tresh = cv.threshold(blur, 20, 180, cv.THRESH_BINARY)
    # 80 e 98% si 150
    canny = cv.Canny(tresh,80, 150)
    #cod din laboratorul 6
    max_area = 0
    contours, _ = cv.findContours(canny,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    contours = sorted(contours, key=cv.contourArea, reverse=True)

    max_area = 0

    for i in range(len(contours)):
        if(len(contours[i]) >3):
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                    possible_bottom_right = point
            diff = np.diff(contours[i].squeeze(), axis = 1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left
        
    image_copy = board.copy()
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),2)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),2)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),2)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),2)
    
 
    # 810,810,54,16
    width = 810
    height = 810

    puzzle = np.array([top_left,top_right,bottom_right,bottom_left], dtype = "float32")
    destination_of_puzzle = np.array([[0,0],[width,0],[width,height],[0,height]], dtype = "float32")

    M = cv.getPerspectiveTransform(puzzle,destination_of_puzzle)

    result = cv.warpPerspective(board.copy(), M, (width, height))

    # show_image("grid",result)
    return result

In [16]:
def draw_lines(grid):
    # horizontal lines
    lines_horizontal=[]
    for i in range(0,811,54):
        l=[]
        l.append((0,i))
        l.append((809,i))
        lines_horizontal.append(l)
    # vertical lines  
    lines_vertical=[]
    for i in range(0,811,54):
        l=[]
        l.append((i,0))
        l.append((i,809))
        lines_vertical.append(l)
        
    # draw lines
    img = grid.copy()
    for line in  lines_vertical : 
        cv.line(img, line[0], line[1], (255, 255, 255), 3)
    for line in  lines_horizontal : 
        cv.line(img, line[0], line[1], (255,255, 255), 3)
    
    
    return (img, lines_horizontal, lines_vertical)

In [17]:
def create_configuration(thresh,lines_horizontal,lines_vertical): 

    matrix = np.empty((15,15), dtype='str')
    for i in range(len(lines_horizontal)-1):
        for j in range(len(lines_vertical)-1):
            y_min = lines_vertical[j][0][0]
            y_max = lines_vertical[j + 1][1][0]
            x_min = lines_horizontal[i][0][1] 
            x_max = lines_horizontal[i + 1][1][1]
            patch = thresh[x_min:x_max, y_min:y_max].copy()
            Medie_patch=np.mean(patch)
            # 100 --> prea mare pragul
            if Medie_patch>100:
                matrix[i][j]='x'
            else:
                matrix[i][j]='o'
    return matrix

In [18]:
def get_positions(result):
    
    result = to_hsv(result,extract_pieces_boundary)
    # show_image('result',result)
    result = cv.cvtColor(result,cv.COLOR_BGR2GRAY)
    # show_image('result',result)
    result = cv.medianBlur(result,23)
    # show_image('result',result)
    # inainte era 150 minim
    _, thresh = cv.threshold(result, 150, 255, cv.THRESH_BINARY)
    # show_image('result',thresh)
    thresh = cv.erode(thresh, np.ones((3, 3), np.uint8))
    thesh = cv.dilate(thresh, np.ones((3, 3), np.uint8))
    # show_image('result',thresh)
    
    thresh, lines_horizontal, lines_vertical = draw_lines(thresh)
    # show_image('grid-ul in format binar',thresh)
    
        
    matrix=create_configuration(thresh,lines_horizontal,lines_vertical)
    
    return matrix

In [19]:
def translate_positions(i,j):
    # linii
    linie = i+1
    # coloane
    coloana = chr(j+65)
    return str(linie)+coloana

In [20]:
def task1_pipeline(img,old_configuration):
   
    board = extract_board(img)
    grid = extract_grid(board)

    current_configuration = get_positions(grid)
    current_positions = []
    for i in range(current_configuration.shape[0]):
        for j in range(current_configuration.shape[1]):
            if current_configuration[i,j] == 'x':
                if current_configuration[i,j] != old_configuration[i,j]:
                    current_positions.append(translate_positions(i,j))
                    
    # de adaugat conditie ca daca sunt mai mult de 2 pozitii gasite sa aibe numarul sau cifra ce mai frecventa
    while len(current_positions) > 2:
        current_positions.pop()

    return (current_positions, current_configuration)

Citire a fisierelor si a imaginilor si creeare dataset-ului

In [21]:
import os
import cv2 as cv
#variabila 'path' mentionata in README, trebuie setata la calea catre folderul cu imaginile de test
path = '../evaluare/fake_test'
images = []
files = sorted(os.listdir(path))
dataset = []
sample = []
# variabila 'test' mentionata in README, trebuie lasta la True
test = True
# alpha = 1 # Factor de contrast
# beta = 40 # Factor de luminozitate
fisiere_mutari = []
fisier_mutari = []

if test:
    len_sample = 1
else:
    len_sample = 6
    
for file in files:
    if file.endswith('jpg'):
        image = cv.imread(path+'/'+file)
        # show_image('image',scale_down(0.3,0.3,image))
        # image = cv.convertScaleAbs(image, alpha=alpha, beta=beta)
        # show_image('image',scale_down(0.3,0.3,image))
        # break
        sample.append(image)
        
    if not test:
        if file.endswith('txt') and 'mutari' not in file:
        
            with open(path+'/'+file) as f:
                nr = 0
                for line in f:
                    if nr == 2:
                        sample.append(int(line))
                    else:
                        sample.append(line.split()[0])
                        sample.append(line.split()[1])
                    nr += 1
                    
    if file.endswith('txt') and 'mutari' in file:
        with open(path+'/'+file) as f:
            nr = 1
            for line in f:
                if nr == 21:
                    break
                fisier_mutari.append(line.split()[1])      
                nr += 1
                
    if len(sample) == len_sample:
        dataset.append(sample)
        sample = []
        
    if len(fisier_mutari) != 0:
        fisiere_mutari.append(fisier_mutari)
        fisier_mutari = []
            

In [22]:
def translate_positions_back(position):
    linie = ''
    coloana = ''
    for i in position:
        if i.isdigit():
            linie += i
        else:
            coloana += i
            
    # linii
    linie = int(linie)-1
    # coloane
    coloana = ord(coloana)-65
    
    return (linie,coloana)

In [23]:
def get_candidates(img,predicted_positions):
    grid = extract_grid(extract_board(img.copy()))
  
    poz1 = translate_positions_back(predicted_positions[0])
    poz2 = translate_positions_back(predicted_positions[1])
    # print(poz1,poz2)
    first_candidate = grid[max(0,poz1[0]*54-3):min(809,poz1[0]*54+54+3),max(0,poz1[1]*54-3):min(809,poz1[1]*54+54+3)]
    second_candidate = grid[max(0,poz2[0]*54-3):min(809,poz2[0]*54+54+3),max(0,poz2[1]*54-3):min(809,poz2[1]*54+54+3)]
    first_candidate,second_candidate = to_hsv(first_candidate,try_boundary),to_hsv(second_candidate,try_boundary)
    return (first_candidate,second_candidate)

In [24]:
def get_number(candidate):
    candidate = cv.cvtColor(candidate,cv.COLOR_BGR2GRAY)
    # show_image('o fata de domino',candidate)
    _,candidate = cv.threshold(candidate, 90, 220, cv.THRESH_BINARY)
    candidate = cv.blur(candidate,(3,3))
    
    numar_cercuri = 0
    # 60 param1 --> 96%, minDist = 13
    cercuri = cv.HoughCircles(candidate, cv.HOUGH_GRADIENT, dp=1, minDist=10, param1=60, param2=10, minRadius=3, maxRadius=8)

    # Draw circles on the original image
    if cercuri is not None:
        # Convert the (x, y) coordinates and radius of the circles to integers
        cercuri = np.round(cercuri[0, :]).astype("int")
        for (x, y, r) in cercuri:
            # print('cate un cerc')
            numar_cercuri += 1
            cv.circle(candidate, (x, y), r, (255, 0, 0), 2)

    # show_image('cercurile gasite',candidate)
    return min(6,numar_cercuri),candidate

In [25]:
def incrementeaza_scor(predicted_positions):
    adauga_la_scor = 0
    for pos in predicted_positions:
        if pos in ROMBURI_ALBASTRE:
            adauga_la_scor += 1
        elif pos in ROMBURI_GALBENE:
            adauga_la_scor += 2
        elif pos in ROMBURI_MARO:
            adauga_la_scor += 3
        elif pos in ROMBURI_NEGRE:
            adauga_la_scor += 4
        elif pos in ROMBURI_VERZI:
            adauga_la_scor += 5
            
    return adauga_la_scor

In [26]:
def dubleaza_scor(scor,numar1,numar2):
    if numar1 == numar2:
        scor *= 2
    return scor

In [27]:
def adauga_3_puncte_bonus(numar1,numar2,contor_player):
    if numar1 == contor_player or numar2 == contor_player:
        return 3
    return 0

In [31]:
old_configuration = np.full((15, 15), 'o', dtype=str)
y_pred = []
contor_player1 = 0
contor_player2 = 0
scor_player1 = 0
scor_player2 = 0
runda = 1
joc = 0
ROMBURI_ALBASTRE = {'5E','5K','7E','7K','9E','9K','11E','11K','5G','5I', '11G','11I','6F','6J','10F','10J'}
ROMBURI_GALBENE = {'5C','11C','6D','10D','3E','13E','4F','12F','4J','12J','3K','13K','6L','10L','5M','11M'}
ROMBURI_MARO = {'8A','3B','13B','2C','14C','4D','12D','1H','15H','4L','12L','2M','14M','3N','13N','8O'}
ROMBURI_NEGRE = {'4A','12A','6B','10B','1D','15D','2F','14F','2J','14J','1L','15L','6N','10N','4O','12O'}
ROMBURI_VERZI= {'1A','15A','1O','15O'}
current_player = 1
numere_tabla = [0,1,2,3,4,5,6,0,2,5,3,4,6,2,2,0,3,5,4,1,6,2,4,5,5,0,6,3,4,2,0,1,5,1,3,4,4,4,5,0,6,3,5,4,1,
                3,3,0,0,1,1,2,3,6,3,5,2,1,0,6,6,5,2,1,2,5,0,3,3,5,0,6,1,4,0,6,3,5,1,4,2,6,2,3,1,6,5,6,2,0,
                4,0,1,6,4,4,1,6,6,3,0]
tabla_de_joc = {i: nr for i, nr in enumerate(numere_tabla)}
pozitie_player1 = 0
pozitie_player2 = 0

Rulare efectiva a task-urilor

In [32]:
for i,sample in enumerate(dataset):
    
    img = sample[0]
    if i % 20 == 0:
        runda = 1
        joc += 1
        contor_player1 = 0
        contor_player2 = 0
        old_configuration = np.full((15, 15), 'o', dtype=str)
        
    # task 1
    predicted_positions,new_configuration = task1_pipeline(img,old_configuration)
    
    # task 2
    if len(predicted_positions) == 2:
        candidate1,candidate2 = get_candidates(img.copy(),predicted_positions.copy())
        nr1,candidate1 = get_number(candidate1)
        nr2,candidate2 = get_number(candidate2)
        
    
            
    scor_player1,scor_player2 = 0,0
    scor_de_adaugat = 0
   
  
        
    # task 3
    if runda <= 20:
        current_player = fisiere_mutari[joc-1][runda-1]
        
        if current_player == 'player1':
            
            if contor_player1 != 0:
                scor_player1 += adauga_3_puncte_bonus(nr1,nr2,pozitie_player1)
                
            if contor_player2 != 0:
                scor_player2 += adauga_3_puncte_bonus(nr1,nr2,pozitie_player2)
            
            scor_de_adaugat += incrementeaza_scor(predicted_positions)
            
            scor_de_adaugat = dubleaza_scor(scor_de_adaugat,nr1,nr2)
            
            scor_player1 += scor_de_adaugat
           
           
                
        elif current_player == 'player2':
            
            if contor_player1 != 0:
                scor_player1 += adauga_3_puncte_bonus(nr1,nr2,pozitie_player1)
                
            if contor_player2 != 0:
                scor_player2 += adauga_3_puncte_bonus(nr1,nr2,pozitie_player2)
                
            scor_de_adaugat += incrementeaza_scor(predicted_positions)
            
            scor_de_adaugat = dubleaza_scor(scor_de_adaugat,nr1,nr2)
            
            scor_player2 += scor_de_adaugat
            
            
        runda += 1
        
  
  
    # calculare rezultate (doar pentru train, unde am rezulatele; variabila test este aceeasi cu cea din 
    # celulare pentru citire a datelor)
    if not test:
        if len(predicted_positions) != 2:
            y_pred.append([False,False])
        else:
            y_pred.append([sample[1] == predicted_positions[0],sample[3] == predicted_positions[1]])
        if nr1 is not None and nr2 is not None:
            y_pred[-1].append(int(sample[2]) == nr1)
            y_pred[-1].append(int(sample[4]) == nr2)
        else:
            y_pred[-1].append(False)
            y_pred[-1].append(False)
        if current_player == 'player1':
            y_pred[-1].append(int(sample[5]) == scor_player1)
        elif current_player == 'player2':
            y_pred[-1].append(int(sample[5]) == scor_player2)
        
    
    contor_player1 += scor_player1
    contor_player2 += scor_player2
    pozitie_player1 = tabla_de_joc[contor_player1]
    pozitie_player2 = tabla_de_joc[contor_player2]
    
    
    # scrie solutia in fisier
    if runda <= 10:
        runda_fisier = '0'+str(runda-1)
    else:
        runda_fisier = str(runda-1)
    
    with open(f'{joc}_{runda_fisier}.txt','w') as g:
        g.write(f'{predicted_positions[0]} {nr1}\n')
        g.write(f'{predicted_positions[1]} {nr2}\n')
        if current_player == 'player1':
            g.write(f'{scor_player1}')
        elif current_player == 'player2':
            g.write(f'{scor_player2}')
    
    
        
    old_configuration = new_configuration
    

In [53]:
# true_positives = len([y for y in y_pred if y[0] == True and y[1] == True and y[2] == True and y[3] == True ])
# false_positives = len([y for y in y_pred if y[0] == False or y[1] == False or y[2] == False or y[3] == False or y[4] == False])

In [33]:
# true_positives,false_positives